# Import modules needed

In [1]:
%%time
import sys
sys.path.append('..')

Wall time: 0 ns


In [3]:
%%time
from source.code.clustering_etl import CLUSTERINGETL
from source.code.word_to_vec_transformer import WordToVecTransformer
from source.code.doc_to_vec_transformer import Doc2VecTransformer

Wall time: 0 ns


In [4]:
%%time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline

from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation as LDA

from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

Wall time: 539 ms


Fix one random state across all notebook cells

In [5]:
%%time
random_state = 0

Wall time: 0 ns


# Read data

In [6]:
%%time
clustering_etl = CLUSTERINGETL('../data/datasets/')


INITIALIZING...
INITIALIZATION HAS BEEN COMPLETED


Wall time: 5 ms


In [7]:
%%time
documents, labels_true = clustering_etl.extract_documents(docs_count_per_topic=100)

Documents tokenization: 100%|█████████████████████████████████████████████████████████| 2000/2000 [00:16<00:00, 123.68it/s]


Wall time: 16.9 s


# TF-IDF documents representation

In [ ]:
%%time
pipeline = Pipeline([
    ('vectorize', CountVectorizer()),
    ('inverse', TfidfTransformer(smooth_idf=True))
])

In [ ]:
%%time
tf_idf_documents = pipeline.fit_transform(documents)

In [ ]:
%%time
tf_idf_documents.shape

# LDA documents representation

In [ ]:
%%time
pipeline = Pipeline([
    ('vectorize', CountVectorizer()),
    ('lda', LDA(n_components=100, random_state=random_state, verbose=10, learning_method='batch', n_jobs=-1))
])

In [ ]:
%%time
lda_documents = pipeline.fit_transform(documents)

In [ ]:
%%time
lda_documents.shape

# Word2Vec representation

In [ ]:
%%time
pipeline = Pipeline([
    ('w2v', WordToVecTransformer(100))
])

In [ ]:
%%time
w2v_documents = pipeline.fit_transform(documents)

In [ ]:
%%time
w2v_documents.shape

# K-Means Clustering with known clusters count

## TF-IDF

In [ ]:
%%time
kmeans = KMeans(n_clusters=20, random_state=random_state)

In [ ]:
%%time
labels_tf_idf_pred = kmeans.fit_predict(tf_idf_documents)

## LDA

In [ ]:
%%time
kmeans = KMeans(n_clusters=20, random_state=random_state)

In [ ]:
%%time
labels_lda_pred = kmeans.fit_predict(lda_documents)

## W2V

In [ ]:
%%time
kmeans = KMeans(n_clusters=20, random_state=random_state)

In [ ]:
%%time
labels_w2v_pred = kmeans.fit_predict(w2v_documents)

# Metrics calculation

## With ground-truth labels

### Confusion matrix

In [ ]:
%%time
mat = metrics.confusion_matrix(labels_true, labels_tf_idf_pred)
sns.heatmap(mat.T, square=True, fmt='d')
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

In [ ]:
%%time
mat = metrics.confusion_matrix(labels_true, labels_lda_pred)
sns.heatmap(mat.T, square=True, fmt='d')
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

In [ ]:
%%time
mat = metrics.confusion_matrix(labels_true, labels_w2v_pred)
sns.heatmap(mat.T, square=True, fmt='d')
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

### Adjusted rand score

In [ ]:
%%time
metrics.adjusted_rand_score(labels_true, labels_tf_idf_pred)

In [ ]:
%%time
metrics.adjusted_rand_score(labels_true, labels_lda_pred)

In [ ]:
%%time
metrics.adjusted_rand_score(labels_true, labels_w2v_pred)

### Mutual info score

In [ ]:
%%time
metrics.mutual_info_score(labels_true, labels_tf_idf_pred)

In [ ]:
%%time
metrics.mutual_info_score(labels_true, labels_lda_pred)

In [ ]:
%%time
metrics.mutual_info_score(labels_true, labels_w2v_pred)

### Adjusted mutual info score

In [ ]:
%%time
metrics.adjusted_mutual_info_score(labels_true, labels_tf_idf_pred) 

In [ ]:
%%time
metrics.adjusted_mutual_info_score(labels_true, labels_lda_pred) 

In [ ]:
%%time
metrics.adjusted_mutual_info_score(labels_true, labels_w2v_pred) 

### Normalized mutual info score

In [ ]:
%%time
metrics.normalized_mutual_info_score(labels_true, labels_tf_idf_pred)

In [ ]:
%%time
metrics.normalized_mutual_info_score(labels_true, labels_lda_pred)

In [ ]:
%%time
metrics.normalized_mutual_info_score(labels_true, labels_w2v_pred)

## Without ground-truth labels

### Silhouette Coefficient

In [ ]:
%%time
metrics.silhouette_score(tf_idf_documents, labels_tf_idf_pred, metric='euclidean')

In [ ]:
%%time
metrics.silhouette_score(lda_documents, labels_lda_pred, metric='euclidean')

In [ ]:
%%time
metrics.silhouette_score(w2v_documents, labels_w2v_pred, metric='euclidean')

### Calinski-Harabaz Index

In [ ]:
%%time
metrics.calinski_harabaz_score(tf_idf_documents.toarray(), labels_tf_idf_pred)

In [ ]:
%%time
metrics.calinski_harabaz_score(lda_documents, labels_lda_pred)

In [ ]:
%%time
metrics.calinski_harabaz_score(w2v_documents, labels_w2v_pred)